In [1]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 4.0 MB/s 
     |████████████████████████████████| 1.3 MB 55.8 MB/s 
     |████████████████████████████████| 636 kB 57.5 MB/s 
     |████████████████████████████████| 16.9 MB 515 kB/s 
     |████████████████████████████████| 25.9 MB 5.6 MB/s 
     |████████████████████████████████| 56 kB 3.2 MB/s 
     |████████████████████████████████| 10.4 MB 21.7 MB/s 
     |████████████████████████████████| 1.7 MB 43.9 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 167 kB 60.9 MB/s 
     |████████████████████████████████| 6.8 MB 25.3 MB/s 
     |████████████████████████████████| 2.0 MB 45.0 MB/s 
     |████████████████████████████████| 3.3 MB 61.5 MB/s 
     |████████████████████████████████

In [5]:
import json
from pip._vendor import requests
import numpy as np
import pandas as pd
from pycaret.classification import *

df_labeled = pd.read_csv('/content/Delhi_weather_dataset-labeled.csv')
data = df_labeled.sample(frac=1)
data_unseen = df_labeled.drop(data.index)
data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)
exp_mclf101 = setup(data=data, target='mode', session_id=123)
gbc = create_model('gbc')
tuned_gbc = tune_model(gbc)




,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.00,0.0,1.0000,1.00,1.0000,1.0000,1.0000
1,1.00,0.0,1.0000,1.00,1.0000,1.0000,1.0000
2,1.00,0.0,1.0000,1.00,1.0000,1.0000,1.0000
3,1.00,0.0,1.0000,1.00,1.0000,1.0000,1.0000
4,0.80,0.0,0.6667,0.70,0.7333,0.6429,0.6944
5,0.80,0.0,0.6667,0.70,0.7333,0.6429,0.6944
6,1.00,1.0,1.0000,1.00,1.0000,1.0000,1.0000
7,1.00,1.0,1.0000,1.00,1.0000,1.0000,1.0000
8,1.00,0.0,1.0000,1.00,1.0000,1.0000,1.0000


INFO:logs:create_model_container: 2
INFO:logs:master_model_container: 2
INFO:logs:display_container: 3
INFO:logs:GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.3, loss='deviance', max_depth=5,
                           max_features=1.0, max_leaf_nodes=None,
                           min_impurity_decrease=0.0005,
                           min_impurity_split=None, min_samples_leaf=3,
                           min_samples_split=7, min_weight_fraction_leaf=0.0,
                           n_estimators=160, n_iter_no_change=None,
                           presort='deprecated', random_state=123,
                           subsample=0.45, tol=0.0001, validation_fraction=0.1,
                           verbose=0, warm_start=False)
INFO:logs:tune_model() succesfully completed......................................


In [19]:
def Gradient_Boosting_Classifier(temp,icon):
   data_unseen=pd.DataFrame([['7/7/22',str(temp),str(icon), 'None']],columns=['datetime', 'temp', 'icon', 'mode']) 
   unseen_predictions = predict_model(tuned_gbc, data=data_unseen)
   unseen_predictions.head()
   save_model(tuned_gbc, 'Final GBC Model')
   saved_final_gbc = load_model('Final GBC Model')
   new_prediction = predict_model(saved_final_gbc, data=data_unseen)
   return new_prediction

def pred(p,d):
    x = d.split()
    d=float(x[0])
    if(d<5):return p
    if(5<d<20):
        if(p=='Cycle'):return "Bike"
        return p
    if(20<d<50):
        if(p==("Cycle") or p==("Bike")):return "Car"
        return p
    if(50<d<100):return "Metro"
    return "Stay at home" 


def printVal():
    x1 =input("Enter Source:")
    x2 =input("Enter Destination:")

    url = 'https://api.distancematrix.ai/maps/api/geocode/json?address=' + x1 + '&key=JkYC5Vkg0Ev8ZuYbIyJaGYa39RuVu'
    r = requests.get(url)
    x = r.json()
    slat = x['result'][0]['geometry']['location']['lat']
    slng = x['result'][0]['geometry']['location']['lng']

    url = 'https://api.distancematrix.ai/maps/api/geocode/json?address=' + x2 + '&key=JkYC5Vkg0Ev8ZuYbIyJaGYa39RuVu'
    r = requests.get(url)
    x = r.json()
    dlat = x['result'][0]['geometry']['location']['lat']
    dlng = x['result'][0]['geometry']['location']['lng']

    url = 'https://api.distancematrix.ai/maps/api/distancematrix/json?origins=' + str(slat) + ',' + str(slng) + '&destinations=' + str(dlat) + ',' + str(dlng) + '&key=JkYC5Vkg0Ev8ZuYbIyJaGYa39RuVu'
    r = requests.get(url)
    x = r.json()
    distance = x['rows'][0]['elements'][0]['distance']['text']

    url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/' + str(slat) + ',' + str(slng) + '?key=WBUJLDHTXM3C7PDWVVRYLA6K4'
    r = requests.get(url)
    x = r.json()
    temp = float(x["days"][0]["hours"][8]['temp'])
    temp = (temp - 32) * (5 / 9)
    icon = x["days"][0]["hours"][8]['icon']
    new_prediction = Gradient_Boosting_Classifier(temp,icon)
    p=new_prediction.at[0, 'Label']
    
    
    return pred(p,distance)



In [20]:
v=printVal()

Enter Source:iitd
Enter Destination:new delhi


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.3, loss='deviance', max_depth=5,
                           max_features=1.0, max_leaf_nodes=None,
                           min_impurity_decrease=0.0005,
                           min_impurity_split=None, min_samples_leaf=3,
                           min_samples_split=7, min_weight_fraction_leaf=0.0,
                           n_estimators=160, n_iter_no_change=None,
                           presort='deprecated', random_state=123,
                           subsample=0.45, tol=0.0001, validation_fraction=0.1,
                           verbose=0, warm_start=False), probability_threshold=None, encoded_labels=False, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.CLASSIFICATION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preload

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0,0,0,0,0,0,0


INFO:logs:Initializing save_model()
INFO:logs:save_model(model=GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.3, loss='deviance', max_depth=5,
                           max_features=1.0, max_leaf_nodes=None,
                           min_impurity_decrease=0.0005,
                           min_impurity_split=None, min_samples_leaf=3,
                           min_samples_split=7, min_weight_fraction_leaf=0.0,
                           n_estimators=160, n_iter_no_change=None,
                           presort='deprecated', random_state=123,
                           subsample=0.45, tol=0.0001, validation_fraction=0.1,
                           verbose=0, warm_start=False), model_name=Final GBC Model, prep_pipe_=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
        

Transformation Pipeline and Model Successfully Saved
Transformation Pipeline and Model Successfully Loaded


INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[], target='mode',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric_strateg...
                                            learning_rate=0.3, loss='deviance',
                                            max_depth=5, max_features=1.0,
                                            max_leaf_nodes=

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0,0,0,0,0,0,0


In [21]:
print("You should take:{}".format(v))

You should take:Bike
